In [1]:
from torch.nn.functional import normalize
from GraphAwareNestedCVEvaluationInductive import GraphAwareNestedCVEvaluationInductive
from torch_geometric.datasets import PPI
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from hyperopt import hp
import numpy as np
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
import shap
import torch
from NestedCV import index_to_mask
from xgboost import XGBClassifier, QuantileDMatrix
from IPython.display import clear_output

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
dataset = PPI(root='data/PPI')
# dataset.transform = T.NormalizeFeatures()

In [3]:
# import xgboost
# from torch_geometric.loader import DataLoader

# train_loader = iter(DataLoader(dataset, batch_size=len(dataset)))
# train_data = next(train_loader)

# xgbc = XGBClassifier(tree_method="hist", device="cuda:2")
# Xy = QuantileDMatrix(train_data.x.numpy(), train_data.y.numpy())
# xgbc.fit(dataset[0].x.cpu().numpy(), dataset[0].y.cpu().numpy())
# xgbc.predict_proba(dataset[0].x.cpu().numpy())

In [4]:
def user_function(kwargs):
    return  kwargs["original_features"] + kwargs["summed_neighbors"]
    
class ModelSpace():
    def __init__(self):
        self.space = None
        self.initialize_space()

    def initialize_space(self):
        framework_choices = {
            'hops': [[3]],
            'attention_config': [{'inter_layer_normalize': False,
    'use_pseudo_attention': True,
    'cosine_eps': 0.01,
    'dropout_attn': None}],
            'user_function': [user_function],
        }
         
        self.space = {
            **{key: hp.choice(key, value) for key, value in framework_choices.items()}
        }
        
    def add_choice(self, key, items):
        self.space[key] = hp.choice(key, items)
        
    def add_uniform(self, key, limits: tuple):
        self.space[key] = hp.uniform(key, limits[0], limits[1])
        
    def add_loguniform(self, key, limits: tuple):
        self.space[key] = hp.loguniform(key, np.log(limits[0]), np.log(limits[1]))
        
    def add_qloguniform(self, key, limits, q):
        self.space[key] = hp.qloguniform(key, low=np.log(limits[0]), high=np.log(limits[1]), q=q)

class XGBSpace(ModelSpace):
    def __init__(self):
        super().__init__()

    def get_space(self):
        self.add_choice("booster", ["gbtree"])
        self.add_choice("n_estimators", [i for i in range(200, 1_000, 200)])
        self.add_choice("max_depth", [None, 6])
        # self.add_choice("max_delta_step", [1, 2, 3])
        # self.add_choice("min_child_weight", [None, *list(range(1,5,1))])
        self.add_choice("device", ["cuda:0"])
        self.add_choice("tree_method", ["hist"])
        self.add_choice("verbosity", [0])

        self.add_choice("early_stopping_rounds", [10])
        self.add_choice("eval_metric", ["error"])
        
        self.add_loguniform("eta", (0.05, 0.7))
        self.add_loguniform("reg_lambda", (0.005, 100))
        self.add_loguniform("reg_alpha", (0.005, 100))

        self.add_uniform("subsample", (0.8, 1))
        self.add_uniform("gamma", (0, 0.2))
        self.add_uniform("colsample_bytree", (0.8, 1))
        self.add_uniform("scale_pos_weight", (0.5, 2)) ## TODO: Change to only within 1+-.1?
        return self.space   

In [ ]:
xgb_space = XGBSpace()

store = dict({})

graph_aware_nestedCV_evaluation = GraphAwareNestedCVEvaluationInductive(2, XGBClassifier, dataset, max_evals= 50, parallelism=5) #len(lr_space.get_space().keys())*20
graph_aware_nestedCV_evaluation.nested_cross_validate(5, 5, xgb_space.get_space())

0it [00:00, ?it/s]

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/14 07:29:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _t

In [ ]:
graph_aware_nestedCV_evaluation.nested_inductive_cv.outer_scores.mean(), graph_aware_nestedCV_evaluation.nested_inductive_cv.outer_scores.std()

In [ ]:
graph_aware_nestedCV_evaluation.nested_inductive_cv.outer_scores.mean()

In [ ]:
graph_aware_nestedCV_evaluation.nested_inductive_cv.outer_scores.std()

In [ ]:
graph_aware_nestedCV_evaluation.nested_inductive_cv.inner_scores.mean(), graph_aware_nestedCV_evaluation.nested_inductive_cv.inner_scores.std()

In [ ]:
##When a model is trained with early stopping, there is an inconsistent behavior between native Python interface and sklearn/R interfaces. By default on R and sklearn interfaces, the best_iteration is automatically used so prediction comes from the best model. But with the native Python interface xgboost.Booster.predict() and xgboost.Booster.inplace_predict() uses the full model. Users can use best_iteration attribute with iteration_range parameter to achieve the same behavior. Also the save_best parameter from xgboost.callback.EarlyStopping might be useful.